In [1]:
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import cx_Oracle
import pandas as pd
from sqlalchemy import create_engine
from collections import Counter

import warnings

warnings.filterwarnings("ignore")

In [2]:
class Aggregate_helper:
    def __init__(self, eq_id, lookback_window, major_down_hour, alarm_table):
        self.eq_id = eq_id
        self.lookback_window = lookback_window
        self.alarm_table = alarm_table
        self.major_down_hour = major_down_hour
        
        ## include all status instead since further preprocessing would be performed
        ## make sure that the timeframe table is a subset of both the alarm and status table to compute major down correctly
        time1 = self.alarm_table.iloc[0]["DT_SET"]
        time2 = self.status_table.iloc[0]["TIMESTAMP_START"]
        timeend1 = self.alarm_table.iloc[len(self.alarm_table)-1]["DT_SET"]
        timeend2 = self.status_table.iloc[len(self.status_table)-1]["TIMESTAMP_START"]
        
        # give a 3 days window to ensure that the alarm and status are captured fully
        start = (max(time1, time2) + timedelta(days=3)).strftime("%d/%m/%Y") 
        end = min(timeend1, timeend2).strftime("%d/%m/%Y")
        
        self.timeframe_table = self.generate_time(start, end, 3)
        self.major_down_arr = self.major_down(self.timeframe_table, self.status_table, self.major_down_hour, 3600)
        self.aggregated = self.aggregate(self.timeframe_table, self.lookback_window, self.alarm_table, self.status_table)
        self.aggregated_table = pd.concat([self.timeframe_table.reset_index(drop=True), self.aggregated.reset_index(drop=True)], axis=1)
        
    def generate_time(self, start_date:str, end_date:str, hour:int):
        start = datetime.strptime(start_date, '%d/%m/%Y')
        end = datetime.strptime(end_date, '%d/%m/%Y')

        dates = []
        while start<=end:
            row = [start]
            dates.append(row)
            start += timedelta(hours=hour)

        return pd.DataFrame(dates, columns=['TIMESTAMP'])
    
    def query_status(self):
        try:
            oracle_string = "oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}"
            engine = create_engine(
                oracle_string.format(
                    username = 'TFM4CEBERUS',
                    password = 'TFM4CEBERUS',
                    hostname = 'ome-db.bth.infineon.com',
                    port = '1538',
                    database = 'ome'
                    )
                )
        except Exception as e:
            print(str(e))

        query = f"""select EQ_ID, TIMESTAMP_START, TIMESTAMP_END, DURATION, STATE_NAME, LEVEL3_NAME, LEVEL3 
                from (SELECT
                  eq.eq_id, eq.name, eq.eq_type_ident
                , data.timestamp_start,data.timestamp_end
                , ROUND((data.timestamp_end - data.timestamp_start)*24*60*60,0) AS Duration
                , data.tr25_3_status,data.tr25_4_status,data.tr25_5_status,data.eq_status
                , level5s.state_name
                , level5.state_name Level5_Name, level5.state_sign Level5
                , level4.state_name Level4_Name, level4.state_sign Level4
                , level3.state_name Level3_Name, level3.state_sign Level3
                ,mh.device
                ,mh.package,
                mh.lotid as lot,
                mh.product,
                mh.operation

                FROM OMEDATA.EQUIPMENT_STATE_HISTORY data
                , OMEADMIN.EQUIPMENT_INSTANCES eq
                , V_EQ_STATES level5s
                , OMEADMIN.DEF_STANDARD_STATEMODEL level5
                , OMEADMIN.DEF_STANDARD_STATEMODEL level4
                , OMEADMIN.DEF_STANDARD_STATEMODEL level3
                , OMEDATA.METAKEY_HISTORY mh

                WHERE data.eq_ident  = eq.eq_ident
                AND  data.eq_status = level5s.state_ident(+)
                AND level5.state_ident = data.tr25_5_status
                AND level4.state_ident = data.tr25_4_status
                AND level3.state_ident = data.tr25_3_status
                AND  data.metakey_ident =mh.ident(+)
                and data.timestamp_start > sysdate - 1050)
                where eq_id = '{self.eq_id}'
                ORDER BY TIMESTAMP_START"""

        status = pd.read_sql(query, engine)
        status.columns = map(lambda x: str(x).upper(), status.columns) 

        return status
    
    def aggregate(self, timeframe_table, lookback_window, alarm_table, status_table):
        alarm_df = pd.DataFrame()
        statename_df = pd.DataFrame()

        for idx, row in timeframe_table.iterrows():
            end = row["TIMESTAMP"]
            start = end - timedelta(hours=lookback_window)

            ## count the frequencies of each alarm
            filtered_alarm = alarm_table.loc[(alarm_table["DT_SET"] >= start) & (alarm_table["DT_SET"] <= end)]
            alarm_freq_table = filtered_alarm["Alarm ID"].value_counts().to_frame().T.reset_index(drop=True)
            alarm_df = pd.concat([alarm_df, alarm_freq_table], axis=0)

            ## count the frequencies of each statename, include everything since feature engineering would be performed
            filtered_statename = status_table.loc[(status_table["TIMESTAMP_START"] >= start) & (status_table["TIMESTAMP_START"] <= end)]
            status_freq_table = filtered_statename["STATE_NAME"].value_counts().to_frame().T.reset_index(drop=True)
            statename_df = pd.concat([statename_df, status_freq_table], axis=0)
        
        df = pd.concat([alarm_df, statename_df], axis=1) # current dataframe
        df = df.fillna(0)

        ## convert all columns from float to int
        cols = df.columns
        df[cols] = df[cols].astype(int)
        df["EQUIPMENT"] = self.eq_id
        return df
        
    def major_down(self, input_df, status_table, hour, threshold): 
        hour = pd.Timedelta(hours=hour)
        major_down = []

        for idx, row in input_df.iterrows():
            start = row['TIMESTAMP']
            end = start+hour
            frame = status_table[(status_table['TIMESTAMP_START']>start) & (status_table['TIMESTAMP_START']<end)]
            UD = frame.loc[frame['LEVEL3']=='UDT']

            if len(UD) == 0: #no record within this 6 hours:
                major_down.append(0)
            else:
                time_diff = (UD['TIMESTAMP_END']-UD['TIMESTAMP_START']).dt.seconds
                if any(time_diff>=threshold): #threshold = 3600s
                    major_down.append(1)
                else:
                    major_down.append(0)
        return np.array(major_down)

In [3]:
wba127_alarm = pd.read_excel(f"../../EQ Alarm Analysis/Data/WBA127_FullAlarm.xlsx", engine='openpyxl', usecols = "B,C,D,F,M")

In [4]:
wba127 = Aggregate_helper('WBA127', 24, 6, wba127_alarm)

In [5]:
label = pd.DataFrame(wba127.major_down_arr, columns=['TARGET'])
wba127_aggregate = pd.concat([wba127.aggregated_table, label], axis = 1)
wba127_aggregate

,TIMESTAMP,1,2,3,4,5,6,8,9,10,...,LOADER/UNLOADER PROBLEM,APC Input Violation,Engineering Evaluation,Undefined Waiting,FullStock,MC Banned,Engineering EE,Material Problem,EQUIPMENT,TARGET
0,2018-12-17 00:00:00,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,WBA127,0
1,2018-12-17 03:00:00,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,WBA127,0
2,2018-12-17 06:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
3,2018-12-17 09:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
4,2018-12-17 12:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7892,2021-08-29 12:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
7893,2021-08-29 15:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
7894,2021-08-29 18:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0
7895,2021-08-29 21:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,WBA127,0


In [6]:
label.value_counts()

TARGET
0         7128
1          769
dtype: int64

In [7]:
def aggregate_tolist(timeframe_table, lookback_window, alarm_table, status_table):
    alarm_df = pd.DataFrame(columns=['Timestamp', 'Alarm List'])
    statename_df = pd.DataFrame(columns=['Timestamp', 'State List'])

    for idx, row in timeframe_table.iterrows():
        end = row["TIMESTAMP"]
        start = end - timedelta(hours=lookback_window)

        ## count the frequencies of each alarm
        filtered_alarm = alarm_table.loc[(alarm_table["DT_SET"] >= start) & (alarm_table["DT_SET"] <= end)]
        alarm_list = filtered_alarm["Alarm ID"].to_list()
        alarm_df.at[idx,'Timestamp'] = end
        alarm_df.at[idx,'Alarm List'] = alarm_list

        ## count the frequencies of each statename, include everything since feature engineering would be performed
        filtered_statename = status_table.loc[(status_table["TIMESTAMP_START"] >= start) & (status_table["TIMESTAMP_START"] <= end)]
        status_list = filtered_statename["STATE_NAME"].to_list()
        statename_df.at[idx, 'Timestamp'] = end
        statename_df.at[idx, 'State List'] = status_list      
        
    df = pd.concat([alarm_df, statename_df], axis=1) # current dataframe

    return df

In [8]:
wba127_corpus = aggregate_tolist(wba127.timeframe_table, 24, wba127.alarm_table, wba127.status_table)

In [9]:
wba127_corpus

,Timestamp,Alarm List,Timestamp,State List
0,2018-12-17 00:00:00,"[46, 73, 46, 46, 46, 51, 11, 73, 46, 46, 46, 4...",2018-12-17 00:00:00,"[SHORT TAIL, Normal Production, Waiting For Re..."
1,2018-12-17 03:00:00,"[46, 44, 46, 46, 11, 10, 46, 46, 46, 46, 46, 7...",2018-12-17 03:00:00,"[SHORT TAIL, Non LIFTED STICK BOND, Normal Pro..."
2,2018-12-17 06:00:00,"[46, 46, 46, 46, 46, 73, 46, 28, 28, 46, 28, 2...",2018-12-17 06:00:00,"[PC Conform, Waiting For Operator, Normal Prod..."
3,2018-12-17 09:00:00,"[28, 46, 28, 28, 28, 28, 28, 46, 46, 46, 46, 4...",2018-12-17 09:00:00,"[Waiting For Operator, Normal Production, SHOR..."
4,2018-12-17 12:00:00,"[45, 44, 44, 44, 28, 28, 28, 28, 23, 28, 28, 2...",2018-12-17 12:00:00,"[Normal Production, Non LIFTED STICK BOND, Nor..."
...,...,...,...,...
7892,2021-08-29 12:00:00,"[11, 27, 46, 27, 46, 28, 46, 46, 28, 46, 46, 1...",2021-08-29 12:00:00,"[Waiting For Response, Normal Production, Norm..."
7893,2021-08-29 15:00:00,"[46, 27, 46, 28, 46, 46, 28, 46, 46, 150, 151,...",2021-08-29 15:00:00,"[SHORT TAIL, Normal Production, Waiting For Re..."
7894,2021-08-29 18:00:00,"[151, 46, 46, 46, 46, 46, 46, 28, 46, 150, 151...",2021-08-29 18:00:00,"[Normal Production, Waiting For Response, Norm..."
7895,2021-08-29 21:00:00,"[46, 28, 46, 150, 151, 46, 28, 46, 150, 151, 1...",2021-08-29 21:00:00,"[SHORT TAIL, Waiting For Operator, Normal Prod..."


In [10]:
wba127_alarm_list = wba127.alarm_table['Alarm ID'].value_counts()
wba127_alarm_list.keys()
wba127_state_list = wba127.status_table['STATE_NAME'].value_counts()
wba127_state_list.keys()

Index(['Normal Production', 'Waiting For Response', 'SHORT TAIL',
       'Waiting For Operator', 'Non LIFTED STICK BOND', 'NSOP/NSOL',
       'NO BONDING', 'Visual Inspection', 'Change Capillary', 'Change Wire',
       'WIRE BREAK', 'Surveillance', 'Waiting For Repair', 'INDEXER PROBLEM',
       'PC Buyoff', 'Change Device', 'Change Lot', 'No Material', 'PlanIdle',
       'Fail Surveillance', 'PC Buyoff Passed', 'Temperature Checking',
       'NoWIP', 'Waiting For Technician', 'Waiting for Setup',
       'POOR RECOGNITION', 'HANG UP', 'PC Conform', 'Machine Failure',
       'Engineering PRE', 'PC Buyoff Failed', 'Non Schedule Time',
       'Change Magazine', 'No Operator', 'BAD WEDGE FORM', 'GOLF BALL BOND',
       'LIFTED BALL BOND', 'Meeting', 'Engineering ME', 'Engineering For Sale',
       'IT Problem', 'BALL PLACEMENT', 'BROKEN WIRE', 'IT Maintenance',
       'Utility Problem', 'Half Yearly PM', 'Engineering EE', 'MOTOR ERROR',
       'WIRE/BALL SIZE', 'Rework/Retest', 'Maintenanc

In [11]:
#calculate number of document that alarm and state happened in
df_alarm = {}
for al in wba127_alarm_list.keys():
    count = 0
    for doc in wba127_corpus['Alarm List']:
        if al in doc:
            count += 1
    if count == 0:
        count = len(wba127_corpus['Alarm List'])
    df_alarm[al] = count
    
df_state = {}
for st in wba127_state_list.keys():
    count = 0
    for doc in wba127_corpus['State List']:
        if al in doc:
            count += 1
    if count == 0:
        count = len(wba127_corpus['State List'])
    df_alarm[al] = count

In [12]:
#calculate idf for alarm and state
from math import log
idf_alarm = {}
idf_state = {}
lenth = len(wba127_corpus['Alarm List'])

for al in df_alarm.keys():
    idf = log(lenth/df_alarm[al], 2)
    idf_alarm[al] = idf
    
for st in df_state.keys():
    idf = log(lenth/df_state[st], 2)
    idf_state[st] = idf

In [13]:
#apply idf value into dataframe, time with the tf of alarm and state
wba127_agg_tfidf = wba127_aggregate.copy()
column = wba127_agg_tfidf.columns
for col in column:
    print(col)
    if col in idf_alarm.keys():
        wba127_agg_tfidf[col] = wba127_agg_tfidf[col] * idf_alarm[col]
    elif col in idf_state.keys():
        wba127_agg_tfidf[col] = wba127_agg_tfidf[col] * idf_state[col]

TIMESTAMP
1
2
3
4
5
6
8
9
10
11
12
13
14
18
19
20
23
24
25
26
27
28
31
33
34
35
44
45
46
47
49
50
51
61
65
73
80
89
90
100
146
147
150
151
245
Normal Production
SHORT TAIL
Waiting For Operator
Waiting For Response
Non LIFTED STICK BOND
POOR RECOGNITION
NO BONDING
WIRE BREAK
NoWIP
Fail Surveillance
PC Conform
Change Device
Change Capillary
Visual Inspection
Surveillance
Change Lot
Machine Failure
Engineering PRE
Non Schedule Time
No Material
INDEXER PROBLEM
BALL PLACEMENT
Change Wire
HANG UP
No Operator
NSOP/NSOL
GOLF BALL BOND
PlanIdle
Waiting For Repair
Waiting For Technician
NO BALL SIZE
WIRE/BALL SIZE
Engineering ME
MALFORM BOND
Change Magazine
BROKEN WIRE
Half Yearly PM
LIFTED BALL BOND
TIGHT WIRE
Waiting For Spares
PC Buyoff
PC Buyoff Passed
Utility Problem
PC Buyoff Failed
IT Maintenance
BAD WEDGE FORM
Meeting
STITCH BOND PLACEMENT
Temperature Checking
Engineering For Sale
Rework/Retest
IT Problem
MOTOR ERROR
5S
Maintenance
Waiting for Setup
LOADER/UNLOADER PROBLEM
APC Input Viol

In [14]:
wba127_agg_tfidf.head(10)[[11, 12, 13, 44,45,46]]

,11,12,13,44,45,46
0,1.862167,0.000000,0.0,6.989109,9.353683,20.517731
1,2.327709,1.669220,0.0,8.299567,9.353683,17.953014
2,2.327709,1.669220,0.0,9.173205,9.903900,17.953014
3,2.793251,1.669220,0.0,9.610025,9.903900,21.372636
4,3.258792,1.669220,0.0,8.736386,1.650650,11.113771
5,3.258792,1.669220,0.0,7.425928,1.100433,11.113771
6,3.724334,1.669220,0.0,6.989109,1.100433,11.113771
7,3.724334,1.669220,0.0,6.115470,1.100433,12.396129
8,3.724334,3.338441,0.0,5.678651,0.550217,12.396129
9,3.258792,3.338441,0.0,3.931374,0.550217,12.396129


In [15]:
# training data using lightgba
import lightgbm
data = wba127_agg_tfidf.drop(['TIMESTAMP', 'EQUIPMENT', 'TARGET'], axis=1)
target = wba127_agg_tfidf['TARGET']

train_idx = int(0.7*len(data))
val_idx = int(0.8*(len(data)))

X_train, y_train = data[:train_idx], target[:train_idx]
X_val, y_val = data[train_idx:val_idx], target[train_idx:val_idx]
X_test, y_test = data[val_idx:], target[val_idx:]

clf = lightgbm.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    random_state=42,
    class_weight='balanced',
    reg_lambda=0.01
            )

clf.fit(X_train, y_train,
       eval_set=(X_val, y_val),
       eval_metric=['auc','aucpr'],
        verbose=False)

pred = clf.predict(X_test)
test_score = accuracy_score(y_test, pred)
training_score = clf.score(X_train, y_train)
pred = clf.predict(X_test)
print(f'Training score: {training_score}, test score: {test_score}')

confusion_matrix(y_test, pred), accuracy_score(y_test, pred)

Training score: 0.959290754478017, test score: 0.779113924050633


(array([[1209,  173],
        [ 176,   22]]),
 0.779113924050633)

In [16]:
# training data using xgboost
import xgboost as xgb
data = wba127_agg_tfidf.drop(['TIMESTAMP', 'EQUIPMENT', 'TARGET'], axis=1)
target = wba127_agg_tfidf['TARGET']
X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size = 0.2, stratify = target, random_state = 10) 
oversample = SMOTE()
X_train_smote, Y_train_smote = oversample.fit_resample(X_train, Y_train)
xgb_ = xgb.XGBClassifier(objective = 'binary:logistic',
                        seed = 40)

xgb_.fit(X_train_smote, 
       Y_train_smote,
       verbose = True,
#       early_stopping_rounds = 20,
       eval_metric = ['auc','aucpr'],
       eval_set = [(X_test, Y_test)])

pred = xgb_.predict(X_test)
test_score = accuracy_score(Y_test, pred)
training_score = clf.score(X_train_smote, Y_train_smote)
pred = xgb_.predict(X_test)
print(f'Training score: {training_score}, test score: {test_score}')

confusion_matrix(Y_test, pred), accuracy_score(Y_test, pred)

[0]	validation_0-auc:0.60017	validation_0-aucpr:0.14118
[1]	validation_0-auc:0.65680	validation_0-aucpr:0.16266
[2]	validation_0-auc:0.67150	validation_0-aucpr:0.18766
[3]	validation_0-auc:0.67914	validation_0-aucpr:0.18709
[4]	validation_0-auc:0.67050	validation_0-aucpr:0.19230
[5]	validation_0-auc:0.67781	validation_0-aucpr:0.19383
[6]	validation_0-auc:0.68258	validation_0-aucpr:0.19479
[7]	validation_0-auc:0.68900	validation_0-aucpr:0.19476
[8]	validation_0-auc:0.69373	validation_0-aucpr:0.20823
[9]	validation_0-auc:0.69458	validation_0-aucpr:0.21236
[10]	validation_0-auc:0.69986	validation_0-aucpr:0.21701
[11]	validation_0-auc:0.70042	validation_0-aucpr:0.22522
[12]	validation_0-auc:0.70237	validation_0-aucpr:0.22987
[13]	validation_0-auc:0.71110	validation_0-aucpr:0.23575
[14]	validation_0-auc:0.70983	validation_0-aucpr:0.23295
[15]	validation_0-auc:0.71056	validation_0-aucpr:0.23383
[16]	validation_0-auc:0.70784	validation_0-aucpr:0.23490
[17]	validation_0-auc:0.70660	validation_

(array([[1376,   50],
        [ 114,   40]]),
 0.8962025316455696)

# Add in Features based on Time

In [17]:
wba127_time = wba127_agg_tfidf.copy()
wba127_time['Weekday'] = [r.date().weekday() for r in wba127_time['TIMESTAMP']]

In [18]:
wba127_time['Weekday']

0       0
1       0
2       0
3       0
4       0
       ..
7892    6
7893    6
7894    6
7895    6
7896    0
Name: Weekday, Length: 7897, dtype: int64

In [19]:
# training data using lightgba
import lightgbm
data = wba127_time.drop(['TIMESTAMP', 'EQUIPMENT', 'TARGET'], axis=1)
target = wba127_time['TARGET']

train_idx = int(0.7*len(data))
val_idx = int(0.8*(len(data)))

X_train, y_train = data[:train_idx], target[:train_idx]
X_val, y_val = data[train_idx:val_idx], target[train_idx:val_idx]
X_test, y_test = data[val_idx:], target[val_idx:]

clf = lightgbm.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    random_state=42,
    class_weight='balanced',
    reg_lambda=0.01
            )

clf.fit(X_train, y_train,
       eval_set=(X_val, y_val),
       eval_metric=['auc','aucpr'],
        verbose=False)

pred = clf.predict(X_test)
test_score = accuracy_score(y_test, pred)
training_score = clf.score(X_train, y_train)
pred = clf.predict(X_test)
print(f'Training score: {training_score}, test score: {test_score}')

confusion_matrix(y_test, pred), accuracy_score(y_test, pred)

Training score: 0.9614619142391895, test score: 0.8082278481012658


(array([[1251,  131],
        [ 172,   26]]),
 0.8082278481012658)

# Using clean status, precise label

In [24]:
store -r wba127_clean_status

In [30]:
wba127_clean_status

,EQ_ID,TIMESTAMP_START,TIMESTAMP_END,DURATION,STATE_NAME,LEVEL3_NAME,LEVEL3
0,WBA127,2018-12-11 12:19:19,2018-12-11 12:34:08,889.0,Normal Production,Productive (PR),PRD
1,WBA127,2018-12-11 12:34:08,2018-12-11 12:37:00,172.0,Waiting For Response,Standby (SB),SBY
2,WBA127,2018-12-11 12:37:00,2018-12-11 13:13:04,2164.0,Normal Production,Productive (PR),PRD
3,WBA127,2018-12-11 13:13:04,2018-12-11 13:18:49,345.0,Waiting For Response,Standby (SB),SBY
4,WBA127,2018-12-11 13:18:49,2018-12-11 13:27:24,515.0,Normal Production,Productive (PR),PRD
...,...,...,...,...,...,...,...
132995,WBA127,2021-10-22 13:35:50,2021-10-22 13:35:51,1.0,Waiting For Response,Standby (SB),SBY
132996,WBA127,2021-10-22 13:35:51,2021-10-22 13:37:17,86.0,Normal Production,Productive (PR),PRD
132997,WBA127,2021-10-22 13:37:17,2021-10-22 13:37:49,32.0,Waiting For Response,Standby (SB),SBY
132998,WBA127,2021-10-22 13:37:49,2021-10-22 15:51:47,8038.0,Normal Production,Productive (PR),PRD


In [25]:
# According to the ratio most of the waiting are minor down, remove all
# 'Waiting For Setup'/'Waiting for Repair' and 'Waiting for Technician' takes up noticable amount of major down
def new_major_down(timeframe_table, wba127_clean_status):
    """
    Clean Status will have all 'Exception' LEVEL3 REMOVED to be able to use the str.contains pandas filter
    Does not affect Major Down count since only EXC is removed
    """
    major_down = []
    for idx, row in timeframe_table.iterrows():
        start = row['TIMESTAMP']
        end = start + timedelta(hours=6)
        
        down_query_filter = ((wba127_clean_status['TIMESTAMP_START']>=start) &
                    (wba127_clean_status['TIMESTAMP_START']<=end) &
                    (~(wba127_clean_status['STATE_NAME'].str.contains('Waiting'))) &
                    (wba127_clean_status['LEVEL3']=='UDT'))
        
        UD = wba127_clean_status[down_query_filter]

        if len(UD) == 0: #no record within this 6 hours:
            major_down.append(0)
        else:
            time_diff = (UD['TIMESTAMP_END']-UD['TIMESTAMP_START']).dt.seconds
            if any(time_diff>=3600): #threshold = 3600s
                major_down.append(1)
            else:
                major_down.append(0)
    return np.array(major_down)

In [32]:
# Remove EXC Level3 state from clean status table
query_filter = (~(wba127_clean_status['LEVEL3']=='EXC'))
wba127_status_noEXC = wba127_clean_status[query_filter]

In [34]:
new_label = new_major_down(wba127.timeframe_table, wba127_status_noEXC)

In [44]:
new_label = pd.DataFrame(new_label, columns=['TARGET'])

In [41]:
def aggregate(timeframe_table, lookback_window, alarm_table, status_table):
    alarm_df = pd.DataFrame()
    statename_df = pd.DataFrame()

    for idx, row in timeframe_table.iterrows():
        end = row["TIMESTAMP"]
        start = end - timedelta(hours=lookback_window)

        ## count the frequencies of each alarm
        filtered_alarm = alarm_table.loc[(alarm_table["DT_SET"] >= start) & (alarm_table["DT_SET"] <= end)]
        alarm_freq_table = filtered_alarm["Alarm ID"].value_counts().to_frame().T.reset_index(drop=True)
        alarm_df = pd.concat([alarm_df, alarm_freq_table], axis=0)

        ## count the frequencies of each statename, include everything since feature engineering would be performed
        filtered_statename = status_table.loc[(status_table["TIMESTAMP_START"] >= start) & (status_table["TIMESTAMP_START"] <= end)]
        status_freq_table = filtered_statename["STATE_NAME"].value_counts().to_frame().T.reset_index(drop=True)
        statename_df = pd.concat([statename_df, status_freq_table], axis=0)
        
    df = pd.concat([alarm_df, statename_df], axis=1) # current dataframe
    df = df.fillna(0)

    ## convert all columns from float to int
    cols = df.columns
    df[cols] = df[cols].astype(int)
    df["EQUIPMENT"] = 'WBA127'
    return df

In [42]:
wba127_clean = aggregate(wba127.timeframe_table, 24, wba127.alarm_table, wba127_status_noEXC)

In [45]:
wba127_agg_new = pd.concat([wba127.timeframe_table.reset_index(drop=True), wba127_clean.reset_index(drop=True), new_label.reset_index(drop=True)], axis=1)

In [49]:
wba127_agg_new['Weekday'] = [r.date().weekday() for r in wba127_agg_new['TIMESTAMP']]

In [50]:
# training data using lightgba
import lightgbm
data = wba127_agg_new.drop(['TIMESTAMP', 'EQUIPMENT', 'TARGET'], axis=1)
target = wba127_agg_new['TARGET']

train_idx = int(0.7*len(data))
val_idx = int(0.8*(len(data)))

X_train, y_train = data[:train_idx], target[:train_idx]
X_val, y_val = data[train_idx:val_idx], target[train_idx:val_idx]
X_test, y_test = data[val_idx:], target[val_idx:]

clf = lightgbm.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    random_state=42,
    class_weight='balanced',
    reg_lambda=0.01
            )

clf.fit(X_train, y_train,
       eval_set=(X_val, y_val),
       eval_metric=['auc','aucpr'],
        verbose=False)

pred = clf.predict(X_test)
test_score = accuracy_score(y_test, pred)
training_score = clf.score(X_train, y_train)
pred = clf.predict(X_test)
print(f'Training score: {training_score}, test score: {test_score}')

confusion_matrix(y_test, pred), accuracy_score(y_test, pred)

Training score: 0.9782884023882757, test score: 0.9284810126582278


(array([[1465,   35],
        [  78,    2]]),
 0.9284810126582278)